### Instalation des librairies

In [ ]:
#install beautiful soup
!pip install bs4


In [ ]:
#install parser
!pip install lxml

In [ ]:
!pip install html5lib

In [ ]:
!pip install requests

In [ ]:
#packages pour scrapper des pages html
import urllib
import bs4
#help(bs4)

In [ ]:
import pandas as pd

### Tests avec la librairie

In [ ]:
#Etape 1 se connecter à la page allocine et obtenir le code source
url_allo_cine= "http://www.allocine.fr/films/"
from urllib import request

request_text = request.urlopen(url_allo_cine).read()
print(request_text[:5000])

In [ ]:
#etape 2 : utiliser la package beautifulSoup qui "comprend" les balises contenues dans la chaine de caractères renvoyée par la fonction request

page = bs4.BeautifulSoup(request_text,"lxml")

print(page)

In [ ]:
print(page.find('meta-title'))

In [ ]:
print(page.find('table'))

In [ ]:
import requests
from requests import get

In [ ]:
#download a page
page = requests.get("http://www.allocine.fr/films/")

In [ ]:
page 

In [ ]:
#print the html content
#page.content

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
#parse the document
soup = BeautifulSoup(page.content, 'lxml')

In [ ]:
#print tje html content of the page formatted nicely
print(soup.prettify())

In [ ]:
#select the elements at the top level of the page
list(soup.children)

In [ ]:
[type(item) for item in list(soup.children)]

In [ ]:
#select the html tag and its children
html = list(soup.children)[1]

In [ ]:
list(html.children)

### scapping allocine

In [ ]:
 for film in soup.find_all('li',{"class":"mdl"}):
    
    #titre film 
    titre_film = film.h2.text
    if titre_film != None:
        print(film.h2.text)
    else :
        titre_film = 'undefined'
        print(titre_film)
 
    #notes:
    
    if film.find_all('span',{"class":"stareval-note"})!= None:
        notes = film.find_all('span',{"class":"stareval-note"})
        if (len(notes))>1 and (notes != None):
            note_presse = notes[0].text
            note_spect = notes[1].text
        elif len(notes) == 1:
            note_presse = notes[0].text
            note_spect = "undefined"
        else:
            note_presse = "undefined"
            note_spect = "undefined"
            
    else:
        note_presse = "undefined"
        note_spect = "undefined"
    print(note_presse)
    print(note_spect)

    #date
    if film.find('span',{"class":"date"})!= None:
        date = film.find('span',{"class":"date"})
    else:
        date = "undefined"
    print(date.text)
    
    
    #duree
    if film.find('span',{"class":"spacer"}).nextSibling != None:
        duree = film.find('span',{"class":"spacer"}).nextSibling
    else:
        duree = "undefined"
        
    print(duree.strip())
    
    #realisateur
    if film.find('a',{"class":"blue-link"}) != None:
        realisateur = film.find('a',{"class":"blue-link"})
    else:
        realisateur = "undefined"
    print(realisateur.text)
        
    #genre
    genres = film.find_all("div", {"class":"meta-body-item meta-body-info"})
    
    list_of_genres=[]
    for genre in genres:
        if len(genre.find_all("span")) < 4:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[2:]]))
        else:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[3:]]))
    
        print(list_of_genres)    
    
    #resume
    resume = film.find('div', {"class":"content-txt"})
    print(resume.text.strip())

In [ ]:
# création dataframe
column_names = ["titre_film", "note_presse", "note_spect", "date_sortie", "duree", "réalisateur", "résumé"]
df = pd.DataFrame(columns = column_names)
# df = df.append(df, ignore_index = True)

In [ ]:
df.head(200)

In [ ]:
df.head(1)

### Mettre le dataframe dans une base de donnée SQL sur Azure

In [ ]:
#install les librairies et drivers 

!pip install pyodbc
! pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine

In [ ]:


AZURESRV = '******.database.windows.net'
AZUREDB = '*******'
AZUREUID = '*******'
AZUREPWD = '****************'
DRIVER= 'ODBC Driver 17 for SQL Server'
TABLE = 'film10'
connectionstring = 'mssql+pyodbc://{uid}:{password}@{server}:1433/{database}?driver={driver}'.format(
        uid=AZUREUID,
        password=AZUREPWD,
        server=AZURESRV,
        database=AZUREDB,
        driver=DRIVER.replace(' ', '+'))



In [ ]:
engn = create_engine(connectionstring)


In [ ]:
df.to_sql(TABLE, engn, if_exists='append', index=False)